# Corrección Tarea 3

A continuación se describe los tests utilizados para corregir la tarea 3.

## Funciones auxiliares

In [1]:
import random

def _is_natural_power(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()
    
    while pow(2, search_exponent) <= n:
        
        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1
                
            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True
            
        search_exponent += 1
    
    return False

In [2]:
def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    r, s, t = _extended_euclid_base(b, a)
    return r, t, s

def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t

In [3]:
def _is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if _is_natural_power(n):
        return False
    
    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False
    
    return found_negative

## La clase Z<sub>p</sub><sup>\*</sup> 

In [4]:
class ZpStar:
    
    def __init__(self, p):
        if not _is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        class Element:
            def __init__(self, value):
                if value < 1 or value > p-1:
                    raise Exception(f"value={value} is not in the range 1,...,{p-1}")
                self.value = value

            # Allows to compare elements with ==
            def __eq__(self, other_element):
                return self.value == other_element.value

            # Allows to operate elements with *
            def __mul__(self, other_element):
                return Element((self.value * other_element.value) % p)

            # Allows to use ** as exponentiation
            def __pow__(self, exponent):
                return Element(pow(self.value, exponent, p))

            # Allows to use str(e) to transform element e into a string
            def __str__(self):
                return str(self.value)
                
        self.element_class = Element
                
    def get_identity(self):
        return self.get_element(1)
    
    def get_element(self, n):
        return self.element_class(n)

## Tests para la clase `ElGamalReceiver`

Para verificar que la clase `ElGamalReceiver` funciona correctamente, se considera la clase `ElGamalReceiver` del alumno y la clase `ElGamalSender` entregada en la [solución de la tarea](https://github.com/UC-IIC3253/2022/tree/main/Tareas/Tarea%203/soluciones/p1).

In [5]:
class ElGamalReceiver:
    def __init__(self, group, generator, group_order):
        # Is the order of the generator correct? For this we check that
        # 1. The group order is prime; and
        # 2. The generator to the power of the order is 1.
        if not _is_probably_prime(group_order):
            raise Exception(f"group_order={group_order} is not a prime number")
        if not generator ** group_order == group.get_identity():
            raise Exception(
                f"group_order={group_order} is not the order of the subgroup"
                f"generated by {generator}"
            )
            
        # The secret key is simply a scalar
        self.secret_key = random.randint(1, group_order - 1)
        
        # The public key must contain the group, the generator,
        # the order of the generator, and the generator to the
        # power secret_key
        self.public_key = (
            group,
            generator,
            group_order,
            generator ** self.secret_key
        )

    def get_public_key(self):
        return self.public_key
        
    def decrypt(self, ciphertext):
        # Returns decryption of ciphertext
        inverse = ciphertext[1] ** (self.public_key[2] - self.secret_key) 
        return (ciphertext[0] * inverse)

In [6]:
class ElGamalSender:
    def __init__(self, pubkey):
        self.group = pubkey[0]
        self.generator = pubkey[1]
        self.group_order = pubkey[2]
        self.pubkey = pubkey[3]

    def encrypt(self, group_element):
        # Definition of the ephemeral key to be used in the encryption
        my_exponent = random.randint(1, self.group_order - 1)
        one_time_pubkey = self.generator ** my_exponent
        
        # In ElGamal the ciphertext is a pair of elements
        return (group_element * (self.pubkey ** my_exponent), one_time_pubkey)        

Para esta combinación de clases se deben realizar los siguientes tests. Se crea el grupo Z<sub>p</sub><sup>\*</sup> considerando valores usados en la práctica, y se encriptan y desencriptan 4 mensajes. Para cada mensaje tal que el resultado de encriptar y desenciptar es el mensaje original, se obtiene 0.375 puntos.

In [7]:
if __name__ == "__main__":
    p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
    generator_value = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
    order = 63762351364972653564641699529205510489263266834182771617563631363277932854227

    group = ZpStar(p)
    generator = group.get_element(generator_value)

    receiver = ElGamalReceiver(group, generator, order)

    sender = ElGamalSender(receiver.get_public_key())

    lista_men = [35,
                 3316599022926473429076949633443971014574932593492,
                 138355476860565497641865052515177602695306881375636627255105412295073965745557843159028438439206280644291186497015498498593710297050242176398959290578210597618476047693783665520812156420333735898905606563332927410345323662066557715149937773259796954529836721732753947908641068730921909553658458711580,
                 581945070195391380113162878258292149229720887277931423233156055445386887864009492416903985525437360974173304731308798129960623491142402265766571630638291434716640588051265938397838657416144582173779876088175146274606946550465692170375119974587379022511472332366260347142927123485513437819824360469314925003354223311798075196031561439474138297963513102089393736835592495782608715457555099865686037050701883565616200153004716354127067558132530951553412459762027988470969769911097544294886132957465250933160185547523226010456849515773261014814360166835304076105227305705493268372409047585521707781875237]
 
    for i in range(4):
        plaintext = group.get_element(lista_men[i])
        ciphertext = sender.encrypt(plaintext)
        dec = receiver.decrypt(ciphertext)
        if (plaintext == dec):
            print("test ",i, " correcto")
        else:
            print("test ",i," incorrecto")
        

test  0  correcto
test  1  correcto
test  2  correcto
test  3  correcto


## Tests para la clase `ElGamalSender`

Para verificar que la clase `ElGamalSender` funciona correctamente, se considera la clase `ElGamalSender` del alumno y la clase `ElGamalReceiver` entregada en la [solución de la tarea](https://github.com/UC-IIC3253/2022/tree/main/Tareas/Tarea%203/soluciones/p1).

In [9]:
class ElGamalReceiver:
    def __init__(self, group, generator, group_order):
        # Is the order of the generator correct? For this we check that
        # 1. The group order is prime; and
        # 2. The generator to the power of the order is 1.
        if not _is_probably_prime(group_order):
            raise Exception(f"group_order={group_order} is not a prime number")
        if not generator ** group_order == group.get_identity():
            raise Exception(
                f"group_order={group_order} is not the order of the subgroup"
                f"generated by {generator}"
            )
            
        # The secret key is simply a scalar
        self.secret_key = random.randint(1, group_order - 1)
        
        # The public key must contain the group, the generator,
        # the order of the generator, and the generator to the
        # power secret_key
        self.public_key = (
            group,
            generator,
            group_order,
            generator ** self.secret_key
        )

    def get_public_key(self):
        return self.public_key
        
    def decrypt(self, ciphertext):
        # Returns decryption of ciphertext
        inverse = ciphertext[1] ** (self.public_key[2] - self.secret_key) 
        return (ciphertext[0] * inverse)

In [10]:
class ElGamalSender:
    def __init__(self, pubkey):
        self.group = pubkey[0]
        self.generator = pubkey[1]
        self.group_order = pubkey[2]
        self.pubkey = pubkey[3]

    def encrypt(self, group_element):
        # Definition of the ephemeral key to be used in the encryption
        my_exponent = random.randint(1, self.group_order - 1)
        one_time_pubkey = self.generator ** my_exponent
        
        # In ElGamal the ciphertext is a pair of elements
        return (group_element * (self.pubkey ** my_exponent), one_time_pubkey)  

Para esta combinación de clases se deben realizar los siguientes tests. Se crea el grupo Z<sub>p</sub><sup>\*</sup> considerando valores usados en la práctica, y se encriptan y desencriptan 4 mensajes. Para cada mensaje tal que el resultado de encriptar y desenciptar es el mensaje original, se obtiene 0.375 puntos.

In [11]:
if __name__ == "__main__":
    p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
    generator_value = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
    order = 63762351364972653564641699529205510489263266834182771617563631363277932854227

    group = ZpStar(p)
    generator = group.get_element(generator_value)

    receiver = ElGamalReceiver(group, generator, order)

    sender = ElGamalSender(receiver.get_public_key())

    lista_men = [652,
                 17087796786602967611503888921121223004118532647751,
                 2769227008904237221523159009531877906413541370582668212122855812934627290578921658134585139139344847484848715757705128437323044411459966279793789889716831837870413422370229874615202060873053154331935545303960586971964224971735509120719958631596634571623491204301242209073520360640059331566827165383861056410600177492497161804279299397966967746043670215078297872101336127503177929218576293653570352693,
                 843650670321255975463146827171249083749788516355722763370497041632347827812006812467438719475283062961241844285355094926155795162268866866094494151165419269451752773051811192284792168490293073670883520672039803351705005366790221950250274012714540991481389953643269702706794308188115804224444989903246234566417111459796779035424745113136083822125925368391029950937517720065441537600846339136146523077556911832356929911222648438226370427349885810604284933462739805763769202036088525530658412266581698263141959256520276048063888229231495162251239799752585325748057533460661952272722978164541942519319931]
    
    for i in range(4):
        plaintext = group.get_element(lista_men[i])
        ciphertext = sender.encrypt(plaintext)
        dec = receiver.decrypt(ciphertext)
        if (plaintext == dec):
            print("test ",i, " correcto")
        else:
            print("test ",i," incorrecto")

test  0  correcto
test  1  correcto
test  2  correcto
test  3  correcto


## Tests para la clase `EllipticCurve`

Para verificiar que la clase `EllipticCurve` funciona correctamente, se considera las clases `ElGamalReceiver` y el `ElGamalSender` de la [solución de la tarea](https://github.com/UC-IIC3253/2022/tree/main/Tareas/Tarea%203/soluciones/p1), y la clase `EllipticCurve` del alumno.

In [12]:
class ElGamalReceiver:
    def __init__(self, group, generator, group_order):
        # Is the order of the generator correct? For this we check that
        # 1. The group order is prime; and
        # 2. The generator to the power of the order is 1.
        if not _is_probably_prime(group_order):
            raise Exception(f"group_order={group_order} is not a prime number")
        if not generator ** group_order == group.get_identity():
            raise Exception(
                f"group_order={group_order} is not the order of the subgroup"
                f"generated by {generator}"
            )
            
        # The secret key is simply a scalar
        self.secret_key = random.randint(1, group_order - 1)
        
        # The public key must contain the group, the generator,
        # the order of the generator, and the generator to the
        # power secret_key
        self.public_key = (
            group,
            generator,
            group_order,
            generator ** self.secret_key
        )

    def get_public_key(self):
        return self.public_key
        
    def decrypt(self, ciphertext):
        # Returns decryption of ciphertext
        inverse = ciphertext[1] ** (self.public_key[2] - self.secret_key) 
        return (ciphertext[0] * inverse)

In [13]:
class ElGamalSender:
    def __init__(self, pubkey):
        self.group = pubkey[0]
        self.generator = pubkey[1]
        self.group_order = pubkey[2]
        self.pubkey = pubkey[3]

    def encrypt(self, group_element):
        # Definition of the ephemeral key to be used in the encryption
        my_exponent = random.randint(1, self.group_order - 1)
        one_time_pubkey = self.generator ** my_exponent
        
        # In ElGamal the ciphertext is a pair of elements
        return (group_element * (self.pubkey ** my_exponent), one_time_pubkey)        

In [14]:
class EllipticCurve:
    def __init__(self, A, B, p):
        if not _is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        elif A < 0 or A > p-1:
            raise Exception(f"A={A} is not in the range 0,...,{p-1}")
        elif B < 0 or B > p-1:
            raise Exception(f"B={B} is not in the range 0,...,{p-1}")
        elif (4*pow(A,3) + 27*pow(B,2)) % p == 0:
            raise Exception(f"The determinant is equal to 0 modulo p={p}")             
        class Element:
            def __init__(self, x, y = None):
                if y is None:
                    if not x == 'N':
                        raise Exception(f"x={x} must take value N")
                    self.x = x
                elif x < 0 or x > p-1:
                    raise Exception(f"x={x} is not in the range 0,...,{p-1}")
                elif y < 0 or y > p-1:
                    raise Exception(f"y={y} is not in the range 0,...,{p-1}")
                elif not (pow(y,2) - (pow(x,3) + A*x +B)) % p == 0: 
                    raise Exception(f"Point x={x}, y={y} is outside the elliptic curve")
                else:
                    self.x = x
                    self.y = y
 

            def __eq__(self, other_element):
                if hasattr(self,'y'):
                    if hasattr(other_element,'y'):
                        return (self.x == other_element.x and self.y == other_element.y)
                    return False
                else:
                    if not hasattr(other_element,'y'):
                        return (self.x == other_element.x)
                    return False

            def __mul__(self, other_element):
                if (self.x == 'N'):
                    return other_element
                elif (other_element.x == 'N'):
                    return self
                elif not self.x == other_element.x:
                    s = ((self.y - other_element.y)*pow(self.x - other_element.x, -1, p)) % p
                    x = (pow(s,2) - self.x - other_element.x) % p
                    y = (s*(self.x - x) - self.y) % p
                    return Element(x,y)
                elif not self.y == other_element.y:
                    return Element('N')
                elif not self.y == 0:
                    s = ((3*pow(self.x, 2) + A)*pow(2*self.y, -1, p)) % p
                    x = (pow(s,2) - self.x - other_element.x) % p
                    y = (s*(self.x - x) - self.y) % p
                    return Element(x,y)
                return Element('N')
                    
            def __pow__(self, exponent):
                if exponent == 0:
                    return Element('N')
                elif exponent == 1:
                    return self
                elif exponent % 2 == 0:
                    t = self ** (exponent//2)
                    return t*t
                t = self ** ((exponent-1)//2)
                return t*t*self

            def __str__(self):
                if hasattr(self,'y'):
                    return "(" + str(self.x) + ", " + str(self.y) + ")"
                return "N"
                    
        self.element_class = Element
                
    def get_identity(self):
        return self.element_class('N')
                    
    def get_element(self, x, y):
        return self.element_class(x,y)

Para esta combinación de clases se deben realizar los siguientes tests. Se crea el grupo para la curva elíptica [P-256](https://neuromancer.sk/std/nist/P-256), y se encriptan y desencriptan 5 mensajes. Para cada mensaje tal que el resultado de encriptar y desenciptar es el mensaje original, se obtiene 0.6 puntos.

In [15]:
if __name__ == "__main__":
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    q = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)
    
    receiver = ElGamalReceiver(group, g, q)

    sender = ElGamalSender(receiver.get_public_key())

    lista_men = [3067470163189886966, 33104927487736301342265473520605062094506791887337775465059167393627219951091,
                 76630183575113934802933426306167335418, 85470266241064418031665052524759502648627664076766195276301173455242207587829,
                 542160151915734992285546605539576536925798487828840674611, 63759637594538567759264190317749821200567023056167818114677286421601349000696,
                 2965894991862942267493820554405407006148122300171087084148788878345692567041, 38276293181344191953842920302338995660105654419446945432484500087352002742649]
    

    for i in range(4):
        plaintext = group.get_element(lista_men[2*i], lista_men[2*i + 1])
        ciphertext = sender.encrypt(plaintext)
        dec = receiver.decrypt(ciphertext)
        if (plaintext == dec):
            print("test ",i, " correcto")
        else:
            print("test ",i," incorrecto")        
            
    plaintext = group.get_identity()
    ciphertext = sender.encrypt(plaintext)
    dec = receiver.decrypt(ciphertext)
        
    if dec == group.get_identity():
        print("test identidad correcto")
    else:
        print("test identidad incorrecto")

test  0  correcto
test  1  correcto
test  2  correcto
test  3  correcto
test identidad correcto
